# Chapter 7 합성곱 신경망(CNN)

## 7.1 전체 구조

CNN에서 등장하는 새로운 layer
- 합성곱 계층(convolutional layer)
- 풀링 계층(pooling layer)

현재까지 봤던 신경망을 완전연결(fully-connected)라고 하며 Affine 계층으로 구현했다. Affine 계층 뒤에는 활성화 함수로 ReLU 혹은 Sigmoid를 사용했다.

CNN의 구조는 새로운 합성곱 계층(Conv)과 풀링 계층(pooling)이 추가된다. Conv-ReLU-(Pooling)의 흐름이다. 출력층과 가까운 층의 경우에는 Affine-ReLU, 출력층은 Affine-Softmax 조합을 그대로 사용한다.

<img src="https://t1.daumcdn.net/cfile/tistory/2409463658F46CAD1F">
<center><small>▲ 완전연결 계층(위), CNN(아래)</small></center>

## 7.2 합성곱 계층

- 패딩(padding)
- 스트라이드(stride)
- 입체적인 데이터 흐름

### 7.2.1 완전연결 계층의 문제점

데이터의 형상이 무시된다.
- 완전연결 계층의 입력은 평탄화 필요
- 본래 다차원인 데이터의 경우 다차원에서 특별한 정보가 담겨있을 가능성이 있다.
- 이미지의 경우
    - 가로, 세로, 색상의 3차원 데이터
    - 공간적 정보
    
반면, 합성곱 계층은 형상을 유지한다.
- 입력 데이터가 형상 그대로 들어온다.
- 다음 계층으로 전달될 때도 그대로 전달
- 다차원의 형상을 가진 데이터를 올바르게 이해할 수 있다.
- 특징 맵(feature map): 합성곱 계층의 입출력 데이터
    - 입력 특징 맵(input feature map)
    - 출력 특징 맵(output feature map)

### 7.2.2 합성곱 연산

합성곱 연산 = 필터 연산


합성곱 연산 예제
<img src="https://t1.daumcdn.net/cfile/tistory/2764173558F475B42C">

데이터 설명
- 입력 데이터: (4,4)의 높이와 너비를 가진 형상
- 필터: (3,3)의 높이와 너비를 가진 형상
    - 커널이라고도 한다.
- 출력: (2,2)의 놆이와 너비를 가진 형상

연산 과정
1. 필터의 **윈도우(window)**를 일정 간격으로 이동하면서 입력 데이터에 적용
2. 단일 곱셈-누산(fused multiply-add, FMA): 대응하는 원소기리 곱한 후 모두 더함
3. 결과를 출력의 해당 장소에 저장
4. 모든 장소에서 수행

가중치와 편향
- 가중치: 필터의 매개변수
- 편향: 필터를 적용한 후 데이터에 더해진다. 항상 하나(1X1)만 존재

### 7.2.3 패딩

패딩(padding)
- 입력 데이터 주변을 특정 값으로 채우는 것
- 예를 들어 0으로

패딩의 예

<img src="https://t1.daumcdn.net/cfile/tistory/2527FA3758F4785B13">

- 입력데이터: 4X4
- 패딩 후: 6X6
- 3X3 필터 적용 후: 4X4

패딩을 하는 이유
- 출력 크기를 조정하는 목적
- 합성곱 신경망에서 그냥 필터를 적용하면 계속해서 크기가 줄어든다. 신경망이 깊어지면 어느 순간 크기가 1이 되어버린다. 이는 더이상 합성곱을 할 수 없는 상태이기 때문에 문제가 된다.
- 풀링을 하면 출력 크기를 유지시켜 줄 수 있어서, 입력 데이터의 공간적 크기를 고정해서 다음 층으로 넘겨줄 수 있다.

### 7.2.4 스트라이드

스트라이드
- 필터를 적용하는 위치 간격
- 스트라이드를 키우면 출력 크기가 작아진다.

<img src="https://2.bp.blogspot.com/-vtZW1-cBQGg/WYJrUnBjRiI/AAAAAAAALNY/GhTnu5QDi3M4NHB_FiyOJAjy58mTkzlYwCK4BGAYYCw/s320/o9.PNG">
<center><small>▲ 스트라이드가 2인 합성곱 신경망</small></center>

패딩, 스트라이드, 출력 크기 계산
- 입력 크기: $(H, W)$
- 필터 크기: $(FH, FW)$
- 출력 크기: $(OH, OW)$
- 패딩: $P$
- 스트라이드: $S$

$$OH = \frac {H + 2P - FH} S + 1$$

$$OW = \frac {W + 2P - FW} S + 1$$

주의
- 계산 결과가 정수로 나누어 떨어져야 한다.

### 7.2.5 3차원 데이터의 합성곱 연산

3차원 데이터의 합성곱 연산 예

<img src="https://t1.daumcdn.net/cfile/tistory/99C185405BC97F4D1E">

- 입력 데이터의 채널 수 = 필터의 채널 수
- 필터의 크기는 원하는 크기로(모든 채널의 필터 크기는 같아야 함)

### 7.2.6 블록으로 생각하기

<img src="https://t1.daumcdn.net/cfile/tistory/998CE7355BC97F632E">

- 데이터와 필터의 형상: (채널, 높이, 너비)
- 출력: 채널이 1개인 특징 맵

<img src="https://t1.daumcdn.net/cfile/tistory/99CDF2395BC97F7C2D">

- 필터를 여러 개 사용하면 출력의 채널 수도 늘어남
- 이 출력을 다음 층으로 넘겨준다.
- 필터의 가중치 데이터는 4차원: (출력 채널 수, 입력 채널 수, 높이, 너비)
- 편향: 채널당 하나의 값

### 7.2.7 배치 처리

<img src="https://t1.daumcdn.net/cfile/tistory/99E4C84E5C4D31B728">

- 4차원으로 데이터 저장: (데이터 수, 채널 수, 높이, 너비)
- 가장 앞쪽에 배치용 차원을 추가
- 각 흐름마다 N번의 합성곱 연산을 수행
- 배치 처리의 효과는 완전연결 신경망과 동일

## 7.3 풀링 계층

풀링 계층
- 가로, 세로 방향의 공간을 줄이는 연산

풀링의 예

<img src="https://t1.daumcdn.net/cfile/tistory/993A5B465C4D32D32C">

- 최대 풀링(max pooling)
    - 대상 영역 내에서 최대값을 구하는 연산
- 평균 풀링(average pooling)
    - 대상 영역의 평균을 계산
- 이미지 인식 분야에서는 최대 풀링을 사용
- 윈도우 크기와 스트라이드는 동일한 값으로 하는 것이 일반적
    - 위의 예: 윈도우 2X2, 스트라이드 2

### 7.3.1 풀링 계층의 특징

학습해야 할 매개변수가 없다.

채널 수가 변하지 않는다.
- 채널마다 독립적으로 계산하기 때문

입력의 변화에 영향을 적게 받는다(강건하다).
- 입력 데이터가 조금 변해도 풀링 결과는 잘 변하지 않는다.

## 7.4 합성곱/풀링 계층 구현하기

### 7.4.1 4차원 배열

CNN에서 흐르는 데이터는 4차원이다.

In [2]:
import numpy as np

In [3]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [4]:
# 첫번째 데이터에 접근
x[0].shape

(1, 28, 28)

In [5]:
# 두번째 데이터에 접근
x[1].shape

(1, 28, 28)

In [6]:
# 첫번째 데이터의 첫 채널에 접근
x[0,0]

array([[9.44322204e-01, 1.61725959e-01, 4.97299575e-01, 4.90849763e-01,
        2.69806956e-01, 4.80739959e-01, 5.33766630e-01, 3.52483845e-01,
        5.90821168e-01, 5.07826272e-01, 6.40826973e-01, 4.59954326e-01,
        5.82373841e-01, 6.52799925e-02, 3.35695899e-01, 4.61650460e-01,
        8.95937799e-01, 6.28431996e-02, 6.36711673e-01, 5.50039978e-01,
        8.44648519e-01, 4.38516371e-01, 5.49044826e-01, 8.53925508e-01,
        1.25532691e-01, 9.38128354e-01, 3.10436399e-01, 6.31229430e-02],
       [5.89551509e-01, 8.42634111e-01, 2.77916824e-01, 5.08458502e-01,
        7.81895554e-01, 6.73924304e-01, 4.29696956e-01, 5.30719806e-01,
        6.09772891e-01, 3.90477187e-01, 8.94129112e-01, 6.94847332e-01,
        1.84154758e-01, 4.93814150e-01, 3.00257947e-01, 7.88172803e-01,
        6.69589655e-01, 2.98000065e-01, 3.52254697e-01, 7.12158128e-01,
        7.26263155e-01, 2.44904846e-02, 8.39847862e-01, 8.55460753e-01,
        6.19307563e-01, 3.10795281e-02, 4.46509670e-01, 7.73360

### 7.4.2 im2col로 데이터 전개하기

넘파이에서 원소를 접근할 때 for 문을 사용하지 않는 것이 바람직하다.

im2col 함수

<img src="https://t1.daumcdn.net/cfile/tistory/99F49B495BA49CE71C">

- 입력 데이터를 필터링(가중치 계산)하기 좋게 펼치는 함수
- 입력 데이터에서 필터를 적용하는 영역을 한 줄로 늘어 놓는다.
- 필터를 적용하는 영역이 겹쳐서 메모리를 더 많이 사용하지만, 선형 대수 라이브러리가 행렬 계산을 매우 빠르게 처리해줘서 속도에서는 이점이 있다.
- Affine 계층에서 한 것과 유사한 계산
- 마지막에 2차원 출력 데이터를 4차원으로 변형(reshape)

### 7.4.3 합성곱 계층 구현하기

In [7]:
# 필터 크기, 스트라이드, 패딩을 고려해서 입력 데이터를 2차원 배열로 전개
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    # np.pad(array, pad_width, mode, **kwargs)
    # array: 패딩할 배열
    # pad_width: 각 축마다 패딩할 값의 수
    # mode: 패딩 방식
    # default로 0으로 패딩
    # 신경망에서 패딩을 하게 되면 이미지와 채널 수에 해당되는 차원은 하지 않기 때문에 pad_width에 해당되는
    # 인수의 처음과 두번째 원소가 (0,0)인 것이다. 즉, 해당 차원은 패딩하지 않는다.
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    # Q. for 문 이해 안됨
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [9]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

In [10]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [14]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [15]:
class Convolution:
    
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.shape(FN, -1). T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

역전파에서는 im2col을 반대로 처리하는 col2im을 사용한다.

In [16]:
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

### 7.4.4 풀링 계층 구현하기

<img src="https://t1.daumcdn.net/cfile/tistory/992F044B5C4D422B02">
<img src="https://t1.daumcdn.net/cfile/tistory/99C888485C4D425402">

합성곱 계층과 마찬가지로 im2col 함수를 이용한다. 다만, 다른 점은 채널마다 독립적으로 전개한다는 점이다.

In [18]:
class Pooling:
    
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 최댓값 (2)
        out = np.max(col, axis=1)
        
        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

## 7.5 CNN 구현하기

손글씨 숫자 인식하는 CNN 구현
- 구조: conv-relu-pooling-affine-relu-affine-softmax

SimpleConvNet 클래스
- \_\_init\_\_
    - 인수
        - input_dim: 입력 데이터(채널 수, 높이, 너비)의 차원
        - conv_param: 합성곱 계층의 하이퍼파라미터(딕셔너리)
            - filter_num: 필터 수
            - filter_size: 필터 크기
            - stride: 스트라이드
            - pad: 패딩
            - hidden_size: 은닉층의 뉴런 수
            - output_size: 출력층의 뉴런 수
            - weight_init_std: 초기화 때의 가중치 표준편차

In [25]:
class SimpleConvNet:
    
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'filter_num':30, 'filter_size':5,
                           'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int((filter_num * (conv_output_size/2) * (conv_output_size/2)))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                          conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.parmas['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.laeyrs['Affine2'] = Affine(self.params['W3'], self.parmas['b3'])
        self.last_layer = SoftmaxWithLoss()
        
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layers.forward(x)
        
        return x
    
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.last_layer.forward(y, t)
    
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine'].db
        
        return grads

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.2995260237545354
=== epoch:1, train acc:0.131, test acc:0.118 ===
train loss:2.2975321648494162
train loss:2.294610007994603
train loss:2.287643742679368
train loss:2.277571750143067
train loss:2.273888829395921
train loss:2.25138555721259
train loss:2.231636338533291
train loss:2.2079077665456355
train loss:2.186551637701792
train loss:2.1616765062348984
train loss:2.1258431776119435
train loss:2.0802510670359355
train loss:2.0478870692285334
train loss:1.9635052471656693
train loss:1.8957273113262747
train loss:1.8494475870518925
train loss:1.7978324885763615
train loss:1.6324522810836712
train loss:1.5907326038955918
train loss:1.5663289527905015
train loss:1.4060911914933985
train loss:1.3127782218226942
train loss:1.4009014144730845
train loss:1.244594319090214
train loss:1.1313555473927368
train loss:1.1168677312278112
train loss:0.9926726162914339
train loss:0.9378282038798514
train loss:1.015293655641714
train loss:0.8153562964995578
train loss:0.7767883218664912
t

train loss:0.20506868751284477
train loss:0.2627372864880542
train loss:0.21828614840433372
train loss:0.3175802711774647
train loss:0.3602783743729605
train loss:0.2220619706003202
train loss:0.21495406083414587
train loss:0.19951440661192651
train loss:0.1450541486713814
train loss:0.1721000697447031
train loss:0.2930998267418376
train loss:0.2570889295961183
train loss:0.12097244924429824
train loss:0.35336132337812665
train loss:0.19814609601867475
train loss:0.23451666428238435
train loss:0.1645692212672131
train loss:0.26659536712542886
train loss:0.22910238472460942
train loss:0.22835879124764855
train loss:0.28316631072734155
train loss:0.25982613191309717
train loss:0.3761664672428233
train loss:0.1539724003366865
train loss:0.32837121749039644
train loss:0.26302596564119823
train loss:0.35589183415847436
train loss:0.285820494399379
train loss:0.2922199229893694
train loss:0.14903603183873698
train loss:0.38072912186858365
train loss:0.25885916380702356
train loss:0.205328720

train loss:0.22352198896871886
train loss:0.21882435157341665
train loss:0.13130392120814444
train loss:0.12691176132323365
train loss:0.1591349820233077
train loss:0.14106356845712328
train loss:0.11827073332728391
train loss:0.12621163529194182
train loss:0.0625820345431352
train loss:0.12101626661145864
train loss:0.0986264335040255
train loss:0.20355673864367468
train loss:0.1520004531006794
train loss:0.13628422587272984
train loss:0.15365474452428649
train loss:0.19043197814695778
train loss:0.1546948801484528
train loss:0.12846829372740076
train loss:0.23166545664080612
train loss:0.14112904662137823
train loss:0.16762794292939226
train loss:0.16597227927324987
train loss:0.17500706217076115
train loss:0.1348197325800523
train loss:0.0675776041558981
train loss:0.21588637695178137
train loss:0.11443338448015322
train loss:0.19095877365341685
train loss:0.15244967571322227
train loss:0.10211084653833716
train loss:0.11218175286405006
train loss:0.1220838068198742
train loss:0.054

train loss:0.12844489332097084
train loss:0.07502702832289738
train loss:0.11089005008166303
train loss:0.10993359084921606
train loss:0.09333240644046523
train loss:0.07249989165397192
train loss:0.06924848126354627
train loss:0.14319087601320885
train loss:0.14318858890208153
train loss:0.10675907386229758
train loss:0.05604820279178284
train loss:0.05292797749304067
train loss:0.08175835750153122
train loss:0.12277845682647713
train loss:0.06759612440762452
train loss:0.11755123667495199
train loss:0.07109392032578085
train loss:0.09652672946643674
train loss:0.027366239958665033
train loss:0.07288582920356902
train loss:0.09301970475243954
train loss:0.09365709208541315
train loss:0.11057604324020176
train loss:0.044464531383469834
train loss:0.039165034695919296
train loss:0.142197119774249
train loss:0.14336121320804526
train loss:0.15545718642655842
train loss:0.18035356821270676
train loss:0.16167867703964314
train loss:0.09835961200317227
train loss:0.06729130536826884
train l

train loss:0.11390339340966314
train loss:0.07404857534110212
train loss:0.14571637643594113
train loss:0.05062364679136491
train loss:0.03787140427960249
train loss:0.06796818335624238
train loss:0.04868135436367213
train loss:0.06034482179977532
train loss:0.07148498976210589
train loss:0.05624147975536669
train loss:0.07546859535450037
train loss:0.051338697021603555
train loss:0.09126782094765547
train loss:0.10711844667679317
train loss:0.10074727835390913
train loss:0.05839706673343348
train loss:0.06415617295295192
train loss:0.08012752354178276
train loss:0.20913424137791575
train loss:0.09495938226367275
train loss:0.055604415105714636
train loss:0.04207244412678464
train loss:0.0330512105282655
train loss:0.0947830071649596
train loss:0.04426318132043344
train loss:0.05271389893172369
train loss:0.05792526344111343
train loss:0.049278774575035315
train loss:0.09414014821777532
train loss:0.04773705518653793
train loss:0.07298338945009085
train loss:0.03673272334846772
train l

...

train loss:0.0014021040717218324
train loss:7.611891499943934e-05
train loss:0.001085080835098025
train loss:0.00011527892509955999
train loss:0.0008349139663734372
train loss:0.0010445541326347906
train loss:0.009387977328797396
train loss:0.00019173416493901152
train loss:0.0001471539072111092
train loss:0.004164818733683191
train loss:0.00037344259586101074
train loss:0.010742738441598778
train loss:0.003676438909684398
train loss:0.005006467828507608
train loss:0.0009770008400258213
train loss:0.0014708499615646476
train loss:0.006130251278452625
train loss:0.00012161683053282213
train loss:0.003491264622004838
train loss:0.0002191263097851786
train loss:0.0012002281795577099
train loss:0.0005096639617698736
train loss:0.019578145365930066
train loss:0.0007753217138348865
train loss:0.0010400395993027797
train loss:0.006576020776403168
train loss:0.000314407804623893
train loss:0.004961700086812252
train loss:7.794224984729604e-05
train loss:0.0009728032957746803
train loss:0.01511

train loss:0.0019025201990544218
train loss:0.0011564196883422973
train loss:0.0023171598092845015
train loss:0.0002747970105336188
train loss:0.0022972436962624197
train loss:0.00039460408219614524
train loss:0.0015366736865839246
train loss:0.002586291831717092
train loss:0.010830283910538452
train loss:0.010584995274219697
train loss:0.001663623553369988
train loss:0.006240782984539376
train loss:0.004760575181392101
train loss:0.0011023608108071655
train loss:0.006610561560701792
train loss:0.0012932649710847921
train loss:0.0007630324059793962
train loss:0.0015397520912207984
train loss:0.007069160470005104
train loss:0.004651426934683936
train loss:0.0031160454678306744
train loss:0.003675743003343386
train loss:0.001153148655749765
train loss:0.0011305899496606984
train loss:0.007389004298421254
train loss:0.0003815579970609832
train loss:0.00028235669916428993
train loss:0.0015491143123198287
train loss:0.0077191241572656995
train loss:0.002979642556427601
train loss:0.00037767

train loss:0.0003802472458610747
train loss:0.0005598894055310927
train loss:0.00027704668143844124
train loss:0.003000930220708302
train loss:0.009688354538006457
train loss:0.0033984234248310556
train loss:0.008619740602872456
train loss:0.0029215657874939022
train loss:0.0005696106928124195
train loss:0.00022989002161309668
train loss:0.0003606748170864036
train loss:0.0012170497375760874
train loss:0.0007712216398018212
train loss:0.03240422779538538
train loss:0.013993210014692281
train loss:0.00612837391594512
train loss:0.00023313951857326516
train loss:0.00028060543802693874
train loss:0.00044894287397229197
train loss:0.0019412328116791206
train loss:0.002636143210124768
train loss:0.001948705883499117
train loss:0.003642833487888099
train loss:0.0016590473622509088
train loss:0.00017497130736101002
train loss:0.00024440379731878914
train loss:0.0011833942475963193
train loss:0.002292325204516588
train loss:0.0016097767131108074
train loss:0.0004335086685236364
train loss:0.00

train loss:0.002375566359560061
train loss:0.0017652310570627047
train loss:0.0015636968130403643
train loss:0.002553266255946617
train loss:0.0037148056248064824
train loss:0.0018162400813962812
train loss:5.209647031043439e-05
train loss:0.000423776109836949
train loss:0.00014580702614703335
train loss:0.00033851612839186357
train loss:0.0008878407395460529
train loss:0.0021287908769842
train loss:0.014385342051676717
train loss:4.1850607200813915e-05
train loss:0.002997614212383856
train loss:0.00208319529724774
train loss:0.000966159895107466
train loss:0.0012790083117528608
train loss:0.010196081707088085
train loss:0.0006425315655453861
train loss:0.0004769174093801902
train loss:0.0030090325819109977
train loss:0.0012327168572504655
train loss:8.252099585166277e-05
train loss:0.0006729391889401955
train loss:0.0034458991494757
train loss:0.0016963677893136898
train loss:0.0004654308896134879
train loss:0.0006209765421302478
train loss:0.002707601744620466
train loss:0.0061974564

train loss:7.26941397053121e-05
train loss:0.0005040294567530875
train loss:0.0011385983421653633
train loss:0.002182041367285257
train loss:0.0028101769144851753
train loss:0.00044807465941932586
train loss:0.00013845979431474428
train loss:0.0006876684312496785
train loss:2.073535034776574e-05
train loss:0.005045253649753096
train loss:0.003095303439391131
train loss:0.001345811561699587
train loss:0.0014412429393835305
train loss:0.0026037475866521847
train loss:0.0010577969166667563
train loss:0.0013496420548710928
train loss:0.002890537866217683
train loss:0.0038759318808511156
train loss:0.00018213983671220156
train loss:0.00215896780401942
train loss:0.00016869926939511496
train loss:1.2433707888303025e-05
train loss:0.0009313235402093543
train loss:0.04356910956297076
train loss:0.002920751638983431
train loss:0.002133962325963391
train loss:0.004167648140819966
train loss:0.00708201706191491
train loss:0.00046374469500508526
train loss:0.001423455357935774
train loss:0.0004656

## 7.6 CNN  시각화하기

### 7.6.1 1번째 층의 가중치 시각화하기

In [ ]:
# coding: utf-8
import numpy as np
import matplotlib.pyplot as plt
from simple_convnet import SimpleConvNet

def filter_show(filters, nx=8, margin=3, scale=10):
    """
    c.f. https://gist.github.com/aidiary/07d530d5e08011832b12#file-draw_weight-py
    """
    FN, C, FH, FW = filters.shape
    ny = int(np.ceil(FN / nx))

    fig = plt.figure()
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    for i in range(FN):
        ax = fig.add_subplot(ny, nx, i+1, xticks=[], yticks=[])
        ax.imshow(filters[i, 0], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()


network = SimpleConvNet()
# 무작위(랜덤) 초기화 후의 가중치
filter_show(network.params['W1'])

# 학습된 가중치
network.load_params("params.pkl")
filter_show(network.params['W1'])

무엇을 보고 있는 것일까?
- 에지(색상이 바뀐 경계선), 블롭(국소적으로 덩어리진 영역) 등
- 예: 세로 에지(필터1)과 가로 에지(필터2)에 반응하는 필터
<img src="https://t1.daumcdn.net/cfile/tistory/999F114D5C4D500610">
- 초기 계층에서 필터는 원시적인 정보를 추출한다.

### 7.6.2 층 깊이에 따른 추출 정보 변화

- 계층이 깊어질수록 추출되는 정보(강하게 반응하는 뉴런)는 더 추상화된다.
- 예: AlexNet
    - 일반 사물 인식 8층 CNN
    - 마지막 층은 완전연결 계층
    - 1층: 에지와 블롭, 3층 텍스쳐, 5층: 사물의 일부, 마지막층: 사물의 클래스에 주로 반응
    - 깊어질 수록 사물의 의미를 이해하도록 변화
    <img src="https://t1.daumcdn.net/cfile/tistory/99AF37505C4D514115">

## 7.7 대표적인 CNN

### 7.7.1 LeNet

<img src="https://t1.daumcdn.net/cfile/tistory/99145F445C4D51F114">

- CNN의 원조
- 손글씨 숫자 인식 네트워크
- 합성곱과 서브 샘플링 계층 반복
    - 현재는 서브 샘플링 대신 최대 풀링
    - 서브 샘플링은 2X2 필터로 average pooling
- 마지막 층: 완전연결 계층
- 활성화 함수: sigmoid
    - 현재는 주로 ReLU 함수 사용

### 7.7.2 AlexNet

<img src="https://t1.daumcdn.net/cfile/tistory/99B4DA3B5C4D52B515">

- 딥러닝 돌풍의 주역
- LeNet과 큰 구조에서는 유사
- 변경점
    - 활성화 함수: ReLU
    - LRN(local response normalization): 국소적 정규화 계층
    - 드롭아웃
    
딥러닝의 발전 원동력은?
- 빅데이터
- GPU